In [4]:
import numpy as np
import cv2
import time


"""
Start of:
Reading input image
"""
image_BGR = cv2.imread('images/original (6).jpg')

cv2.namedWindow('Original Image', cv2.WINDOW_NORMAL)

cv2.imshow('Original Image', image_BGR)

cv2.waitKey(0)

cv2.destroyWindow('Original Image')

print('Image shape:', image_BGR.shape)  


h, w = image_BGR.shape[:2]  

print('Image height={0} and width={1}'.format(h, w))  # 511 767

"""
End of: 
Reading input image
"""


"""
Start of:
Getting blob from input image
"""


blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416),
                             swapRB=True, crop=False)

print('Image shape:', image_BGR.shape)  # (511, 767, 3)
print('Blob shape:', blob.shape)  # (1, 3, 416, 416)


blob_to_show = blob[0, :, :, :].transpose(1, 2, 0)
print(blob_to_show.shape)  # (416, 416, 3)


cv2.namedWindow('Blob Image', cv2.WINDOW_NORMAL)

cv2.imshow('Blob Image', cv2.cvtColor(blob_to_show, cv2.COLOR_RGB2BGR))

cv2.waitKey(0)

cv2.destroyWindow('Blob Image')

"""
End of:
Getting blob from input image
"""


"""
Start of:
Loading YOLO v3 network
"""

with open('yolo-coco-data/coco.names') as f:

    labels = [line.strip() for line in f]


print('List with labels names:')
print(labels)

network = cv2.dnn.readNetFromDarknet('yolo-coco-data/yolov3.cfg',
                                     'yolo-coco-data/yolov3.weights')


layers_names_all = network.getLayerNames()

print()
print(layers_names_all)

layers_names_output = \
    [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]

# Check point
print()
print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']


probability_minimum = 0.5

threshold = 0.3

colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

print()
print(type(colours))  # <class 'numpy.ndarray'>
print(colours.shape)  # (80, 3)
print(colours[0])  # [172  10 127]

"""
End of:
Loading YOLO v3 network
"""


"""
Start of:
Implementing Forward pass
"""

network.setInput(blob)  
start = time.time()
output_from_network = network.forward(layers_names_output)
end = time.time()


print('Objects Detection took {:.5f} seconds'.format(end - start))

"""
End of:
Implementing Forward pass
"""


"""
Start of:
Getting bounding boxes
"""


bounding_boxes = []
confidences = []
class_numbers = []



for result in output_from_network:
    for detected_objects in result:
        scores = detected_objects[5:]
        class_current = np.argmax(scores)

        confidence_current = scores[class_current]


        print(detected_objects.shape)  # (85,)

        if confidence_current > probability_minimum:

            box_current = detected_objects[0:4] * np.array([w, h, w, h])

            x_center, y_center, box_width, box_height = box_current
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)

"""
End of:
Getting bounding boxes
"""


"""
Start of:
Non-maximum suppression
"""


results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                           probability_minimum, threshold)

"""
End of:
Non-maximum suppression
"""


"""
Start of:
Drawing bounding boxes and labels
"""

counter = 1

if len(results) > 0:
    for i in results.flatten():
        print('Object {0}: {1}'.format(counter, labels[int(class_numbers[i])]))

        counter += 1

        x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
        box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

        colour_box_current = colours[class_numbers[i]].tolist()

        print(type(colour_box_current))  # <class 'list'>
        print(colour_box_current)  # [172 , 10, 127]

        cv2.rectangle(image_BGR, (x_min, y_min),
                      (x_min + box_width, y_min + box_height),
                      colour_box_current, 2)

        text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                               confidences[i])

        cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
                    cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)


print()
print('Total objects been detected:', len(bounding_boxes))
print('Number of objects left after non-maximum suppression:', counter - 1)

"""
End of:
Drawing bounding boxes and labels
"""


cv2.namedWindow('Detections', cv2.WINDOW_NORMAL)
cv2.imshow('Detections', image_BGR)
cv2.waitKey(0)
cv2.destroyWindow('Detections')

Image shape: (548, 821, 3)
Image height=548 and width=821
Image shape: (548, 821, 3)
Blob shape: (1, 3, 416, 416)
(416, 416, 3)
List with labels names:
['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drie

(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,

(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,

(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,

(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,

(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,)
(85,

"\nSome comments\n\nWith OpenCV function 'cv2.dnn.blobFromImage' we get 4-dimensional\nso called 'blob' from input image after mean subtraction,\nnormalizing, and RB channels swapping. Resulted shape has:\n - number of images\n - number of channels\n - width\n - height\nE.G.: blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)\n"